In [1]:
import os
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import shutil as sh
import math
import time
from mpl_toolkits.mplot3d import Axes3D
from tqdm import tqdm

FF 関数

In [25]:
def Rod(n,theta_in):
    nx,ny,nz=n
    theta_t=np.radians(theta_in)
    Rod=np.array([[np.cos(theta_t)+(nx**2)*(1-np.cos(theta_t)),nx*ny*(1-np.cos(theta_t))-nz*np.sin(theta_t),nx*nz*(1-np.cos(theta_t))+ny*np.sin(theta_t)],
                [nx*ny*(1-np.cos(theta_t))+nz*np.sin(theta_t),np.cos(theta_t)+(ny**2)*(1-np.cos(theta_t)),ny*nz*(1-np.cos(theta_t))-nx*np.sin(theta_t)],
                [nx*nz*(1-np.cos(theta_t))-ny*np.sin(theta_t),ny*nz*(1-np.cos(theta_t))+nx*np.sin(theta_t),np.cos(theta_t)+(nz**2)*(1-np.cos(theta_t))]])
    return Rod

def get_rot_axis_from_A2(A2):
    A2 = np.radians(A2)
    rot_axis1 = np.array([0.,0.,1.])
    rot_axis2 = np.array([
        -np.sin(A2),
        np.cos(A2),
        0.
    ])
    return rot_axis1, rot_axis2

def get_ab_from_params(R1,R2,heri):
    A_rad=np.radians(heri/2)
    a_=2*(R1*np.cos(A_rad)-R2*np.sin(A_rad))
    b_=2*(R2*np.cos(A_rad)+R1*np.sin(A_rad))
    return a_, b_ 

def getA1_from_R3t(a,R3t,glide):
    assert glide=='a'
    return np.rad2deg(np.arctan(R3t/(a/2)))

#分子を配列
# mono-Cn-BTBTを想定
def arrangeNN(atom_list,a,b,A1,A2,A3,phi1,phi2,glide):

    atom_list_i=[];atom_list_p1=[];atom_list_p2=[]
    atom_list_t1=[];atom_list_t2=[];atom_list_t3=[];atom_list_t4=[]

    #alkylの基準
    C0=np.array([atom_list[16][1],atom_list[16][2],atom_list[16][3]])
    C1=np.array([atom_list[23][1],atom_list[23][2],atom_list[23][3]])

    #phi1に関するalkylの軸
    n1=C1-C0
    n1/=np.linalg.norm(n1)

    #映進面によって回転角変える
    glide=180.0 if glide=='a' else 0.0

    #回転軸
    rot_axis_i1, rot_axis_i2 = get_rot_axis_from_A2(A2)
    rot_axis_t1, rot_axis_t2 = get_rot_axis_from_A2(-A2)

    #alkyl回転・分子1作成
    for ind,(x,y,z,R) in enumerate(atom_list):
        x1=x;y1=y;z1=z
        x2=x;y2=y;z2=z
        #alkylだけΦ回転　phi1=-phi2のとき映進
        if ind>=23:#alkyl?
            x1,y1,z1=np.matmul(Rod(n1,phi1),(np.array([x,y,z])-C0).T)
            x1,y1,z1=C0+np.array([x1,y1,z1])
            x2,y2,z2=np.matmul(Rod(n1,phi2),(np.array([x,y,z])-C0).T)
            x2,y2,z2=C0+np.array([x2,y2,z2])

        #heri/2回転 i,t作成
        x1,y1,z1=np.matmul(Rod(rot_axis_i1,A3 + A2),np.array([x1,y1,z1]).T)
        x1,y1,z1=np.matmul(Rod(rot_axis_i2,A1),np.array([x1,y1,z1]).T)
        
        x2,y2,z2=np.matmul(Rod(rot_axis_t1,-A3 - A2 +glide),np.array([x2,y2,z2]).T)
        x2,y2,z2=np.matmul(Rod(rot_axis_t2,A1),np.array([x2,y2,z2]).T)

        atom_list_i.append([x1,y1,z1,R])
        atom_list_p1.append([x1,b+y1,z1,R])
        atom_list_p2.append([x1,-b+y1,z1,R])
        atom_list_t1.append([a/2+x2,b/2+y2,z2,R]) 
        atom_list_t2.append([a/2+x2,-b/2+y2,z2,R])
        atom_list_t3.append([-a/2+x2,-b/2+y2,z2,R])
        atom_list_t4.append([-a/2+x2,b/2+y2,z2,R])

    xyz_NN=np.array(atom_list_p1+atom_list_p2+atom_list_t1+atom_list_t2+atom_list_t3+atom_list_t4)
    
    return xyz_NN

#mono-BTBT-Cnを想定。
def make_paraview(monomer_filename,a,b,A1,A2,A3,phi1,phi2,glide):
    base_dir = '/home/koyama/Working/interaction/paraview'
    df_mono=pd.read_csv(os.path.join(base_dir,monomer_filename))
    monomer=df_mono[['X','Y','Z','R']].values
    xyz_NN=arrangeNN(monomer,a,b,A1,A2,A3,phi1,phi2,glide)
    df_xyz = pd.DataFrame(data = xyz_NN, columns = ['X','Y','Y','R'])
    isBTBT = monomer_filename == 'BTBT.csv'
    if isBTBT:
        out_filename = 'arrangesNN/a={}_b={}_A1={}_A2={}_A3={}_glide={}_'.format(round(a,2),round(b,2),int(A1),int(A2),int(A3),glide)+monomer_filename
    else:
        out_filename = 'arrangesNN/a={}_b={}_A1={}_A2={}_A3={}_phi1={}_phi2={}_glide={}_'.format(round(a,2),round(b,2),int(A1),int(A2),int(A3),phi1,phi2,glide)+monomer_filename
    df_xyz.to_csv(os.path.join(base_dir,out_filename),index=False)


In [26]:
df_vdw = pd.read_csv('/home/koyama/Working/interaction/BTBT/vdw/vdw_step2B_glide=a_edge=a_0.1deg.csv')
df_vdw[(df_vdw['A1']==6) & (df_vdw['A2']==-11)]

,A1,A2,A3,theta,R,S,a,b
984,6.0,-11.0,41.0,35.7,5.199799,51.251352,8.445342,6.068594


In [27]:
glide='a'
A1,A2,A3,theta,R,S,a,b = df_vdw.iloc[984]
make_paraview('BTBT.csv',a,b,A1,A2,A3,phi1,phi2,glide)